### Install necessary packages

In [1]:
!pip install pandas python-docx
!pip install pywin32 python-docx
!pip install openpyxl

### Import necessary packages

In [2]:
import os
import zipfile
import pandas as pd
from docx import Document
import csv
import os
import re  # Import the regular expression module


In [3]:
# Define the path where the ZIP files are located
zip_files_path = 'C:\\Users\\muc574\\Bid Protest\\WestLaw Data\\Data'
unzip_folders_path = zip_files_path  # Assuming you want to unzip in the same location

# Unzip each ZIP file into a different subfolder
for filename in os.listdir(zip_files_path):
    if filename.endswith('.zip'):
        # Create a subfolder for the contents of this ZIP file
        subfolder_name = os.path.splitext(filename)[0]
        subfolder_path = os.path.join(unzip_folders_path, subfolder_name)
        os.makedirs(subfolder_path, exist_ok=True)
        
        # Unzip the ZIP file into the subfolder
        zip_file_path = os.path.join(zip_files_path, filename)
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(subfolder_path)
        print(f"Unzipped {filename} into {subfolder_path}")

      

Unzipped 1.zip into C:\Users\muc574\Bid Protest\WestLaw Data\Data\1
Unzipped 10.zip into C:\Users\muc574\Bid Protest\WestLaw Data\Data\10
Unzipped 11.zip into C:\Users\muc574\Bid Protest\WestLaw Data\Data\11
Unzipped 12.zip into C:\Users\muc574\Bid Protest\WestLaw Data\Data\12
Unzipped 13.zip into C:\Users\muc574\Bid Protest\WestLaw Data\Data\13
Unzipped 14.zip into C:\Users\muc574\Bid Protest\WestLaw Data\Data\14
Unzipped 15.zip into C:\Users\muc574\Bid Protest\WestLaw Data\Data\15
Unzipped 16.zip into C:\Users\muc574\Bid Protest\WestLaw Data\Data\16
Unzipped 17.zip into C:\Users\muc574\Bid Protest\WestLaw Data\Data\17
Unzipped 18.zip into C:\Users\muc574\Bid Protest\WestLaw Data\Data\18
Unzipped 19.zip into C:\Users\muc574\Bid Protest\WestLaw Data\Data\19
Unzipped 2.zip into C:\Users\muc574\Bid Protest\WestLaw Data\Data\2
Unzipped 20.zip into C:\Users\muc574\Bid Protest\WestLaw Data\Data\20
Unzipped 21.zip into C:\Users\muc574\Bid Protest\WestLaw Data\Data\21
Unzipped 22.zip into C:\

## Data Collection and Analysis

### Step 1 - Read Doc Files into CSV 

In [4]:
##export conclusion sections
import csv
import os
import re

def save_final_clean_text_to_csv(root_dir, csv_filename='final_clean_text.csv'):
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Subdir', 'Filename', 'Final Clean Text'])

        # Cleanup pattern to remove RTF control words, formatting characters, braces, and reduce all kinds of whitespace
        cleanup_pattern = re.compile(r'(\\par|\\[a-z]+\d*|\{\}|\{|\}|\n|\r|\t|\f|\v|\s{2,})')

        for subdir, dirs, files in os.walk(root_dir):
            for filename in files:
                if filename.endswith('.doc'):  # Adjust as necessary for your file types
                    file_path = os.path.join(subdir, filename)
                    try:
                        with open(file_path, 'r', encoding='utf-8') as file:
                            content = file.read()

                            # Find all occurrences of "conclusion" and "All Citations"
                            conclusions = [(m.start(0), m.end(0)) for m in re.finditer('conclusion', content, re.IGNORECASE | re.DOTALL)]
                            all_citations = [(m.start(0), m.end(0)) for m in re.finditer('All Citations', content, re.IGNORECASE | re.DOTALL)]

                            # Determine the last "conclusion" before the last "All Citations", if "All Citations" exists
                            if all_citations:
                                last_citation_start, _ = all_citations[-1]
                                conclusions_before_last_citation = [end for start, end in conclusions if start < last_citation_start]
                                if conclusions_before_last_citation:
                                    conclusion_end = conclusions_before_last_citation[-1]
                                    all_citations_start = last_citation_start
                                    text_after_conclusion_before_citations = content[conclusion_end:all_citations_start].strip()
                                else:
                                    continue  # Skip if no valid conclusion before last "All Citations"
                            else:
                                # If "All Citations" does not exist, use the last "conclusion"
                                if conclusions:
                                    _, conclusion_end = conclusions[-1]
                                    text_after_conclusion_before_citations = content[conclusion_end:].strip()
                                else:
                                    continue  # Skip if no "conclusion" at all

                            # Further cleanup of the text, including reducing excessive whitespace
                            clean_text = re.sub(cleanup_pattern, ' ', text_after_conclusion_before_citations).strip()
                            # Additional step to ensure the text is a coherent paragraph
                            coherent_paragraph = ' '.join(clean_text.split())
                            #print(f"File: {filename}, Final Clean Text: {coherent_paragraph[:100]}...")
                            csvwriter.writerow([subdir, filename, coherent_paragraph])
                    except Exception as e:
                        print(f"Error reading file {filename}: {e}")

# Define the root directory
root_directory = 'C:\\Users\\muc574\\Bid Protest\\WestLaw Data\\Data'

# Execute the function with the root directory as its argument
save_final_clean_text_to_csv(root_directory)

In [5]:
# Path to the directory
root_dir = 'C:\\Users\\muc574\\Bid Protest\\WestLaw Data\\Data'

# Counter for .doc files
doc_file_count = 0

# Walk through the directory and its subdirectories
for subdir, dirs, files in os.walk(root_dir):
    for file in files:
        if file.endswith('.doc'):
            doc_file_count += 1

# Print the number of .doc files
print(f"Number of .doc files under {root_dir}: {doc_file_count}")


Number of .doc files under C:\Users\muc574\Bid Protest\WestLaw Data\Data: 3405


### Step 2 - Convert to XL

In [6]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('final_clean_text.csv')

# Filter rows where 'Subdir' column contains the specified path
filtered_df = df[df['Subdir'].str.contains(r'C:\\Users\\muc574\\Bid Protest\\WestLaw Data\\Data\\.*', regex=True)]

# Save the filtered data to an XLSX file using openpyxl
filtered_df.to_excel('final_clean_text_2.xlsx', index=False, engine='openpyxl')


In [7]:
filtered_df.head()

,Subdir,Filename,Final Clean Text
0,C:\Users\muc574\Bid Protest\WestLaw Data\Data\1,001 - PDS Consultants Inc v United States.doc,"For all of these reasons, the court GRANTS IFB..."
1,C:\Users\muc574\Bid Protest\WestLaw Data\Data\1,002 - Veterans Contracting Group Inc v United ...,"For the reasons stated, Veterans motion for a ..."
2,C:\Users\muc574\Bid Protest\WestLaw Data\Data\1,003 - Q Integrated Companies LLC v United Stat...,"For the foregoing reasons, Q Integrated is awa..."
3,C:\Users\muc574\Bid Protest\WestLaw Data\Data\1,004 - In re Global Computer Enterprises Inc.doc,"For the foregoing reasons, the Court finds in ..."
4,C:\Users\muc574\Bid Protest\WestLaw Data\Data\1,005 - AT And T Corp v United States.doc,"For the foregoing reasons, the Court concludes..."


### Step 3 - Now generate data for excluded files

#### Identify excluded files

In [8]:
import os
import pandas as pd

# Path to the directory
root_dir = 'C:\\Users\\muc574\\Bid Protest\\WestLaw Data\\Data'

# Read the Excel file to get the list of subdirectories already included
df = pd.read_excel('final_clean_text_2.xlsx', engine='openpyxl')
included_subdirs = set(df['Filename'])
print(df['Filename'].shape)

# List to store paths of .doc files not included in the Excel file
excluded_files = []
# List to store paths of .doc files not included in the Excel file
excluded_files_long = []


# Walk through the directory and its subdirectories
for subdir, dirs, files in os.walk(root_dir):
    for file in files:
        if file.endswith('.doc'):
            file_path = os.path.join(subdir, file)
            #print(file)
            # Check if the file's directory is not in the list of included subdirectories
            if file not in included_subdirs:
                excluded_files.append(file)
                excluded_files_long.append(file_path)

print(len(excluded_files))

(2817,)
587


#### Extract info from excluded files

In [9]:
import csv
import os
import re

def clean_and_extract_last_100_chars(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            # Split and remove all text after "All Citations"
            content = re.split('All Citations', content, flags=re.IGNORECASE)[0]
            # Extended cleanup pattern to also remove specific patterns and sequences of spaces
            cleanup_pattern = re.compile(
                r'(\\par|\\[a-z]+\d*|\{\}|\{|\}|\n|\r|\t|\f|\v|\s{2,}|\\[*]co_allCitations_\d+\\[*]co_allCitations_\d+|\s{5,})'
            )
            clean_content = re.sub(cleanup_pattern, '', content).strip()
            return clean_content[-1000:]
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return None

# Prepare to write to a CSV file
with open('additionaldata.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['Subdir', 'Filename', 'Final Clean Text'])

    for file_path in excluded_files_long:
        #print(file_path)
        last_100_chars = clean_and_extract_last_100_chars(file_path)
        if last_100_chars:
            directory, filename = os.path.split(file_path)
            csvwriter.writerow([directory, filename, last_100_chars])


### Step 4 - Merge all data

In [10]:
import pandas as pd
import os

# Read the CSV file
df_csv = pd.read_csv('additionaldata.csv')

# Read the Excel file
df_excel = pd.read_excel('final_clean_text_2.xlsx', engine='openpyxl')

# Append the dataframes
df_combined = df_csv.append(df_excel, ignore_index=True)

# Export the combined dataframe to a new Excel file
df_combined.to_excel('final_clean_text_3.xlsx', index=False, engine='openpyxl')

# Confirm the number of rows in the final dataframe is equal to the sum of the rows in each
print(f"Rows in additionaldata.csv: {df_csv.shape[0]}")
print(f"Rows in final_clean_text_2.xlsx: {df_excel.shape[0]}")
print(f"Rows in combined dataframe: {df_combined.shape[0]}")
assert df_combined.shape[0] == df_csv.shape[0] + df_excel.shape[0], "Row counts do not match."



C:\Users\muc574\AppData\Local\Temp\ipykernel_11452\1100021393.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_combined = df_csv.append(df_excel, ignore_index=True)


Rows in additionaldata.csv: 587
Rows in final_clean_text_2.xlsx: 2817
Rows in combined dataframe: 3404


### Step 5 - Calculate rejection rates

In [11]:
import pandas as pd
import numpy as np
import re

# Read the Excel file
df = pd.read_excel('final_clean_text_3.xlsx', engine='openpyxl')

# Function to check if the patterns "denied" or "sustained" exist in the text
def contains_pattern(text, pattern):
    if pd.isna(text):
        return 0
    return 1 if re.search(pattern, text, re.IGNORECASE) else 0

# Generate dummy variables
df['contains_denied'] = df['Final Clean Text'].apply(lambda x: contains_pattern(x, 'denied'))
df['contains_rejected'] = df['Final Clean Text'].apply(lambda x: contains_pattern(x, 'rejected'))
df['contains_dismissed'] = df['Final Clean Text'].apply(lambda x: contains_pattern(x, 'dismissed'))
df['contains_remanded'] = df['Final Clean Text'].apply(lambda x: contains_pattern(x, 'remanded'))

df['total_unsuccessful'] = df['contains_denied'] + df['contains_rejected'] + df['contains_dismissed'] +df['contains_remanded']

df['contains_settled'] = df['Final Clean Text'].apply(lambda x: contains_pattern(x, 'settled'))

# Optional: save the updated dataframe with dummy variables to a new Excel file
df.to_excel('final_clean_text_4.xlsx', index=False, engine='openpyxl')

df.head()


,Subdir,Filename,Final Clean Text,contains_denied,contains_rejected,contains_dismissed,contains_remanded,total_unsuccessful,contains_settled
0,C:\Users\muc574\Bid Protest\WestLaw Data\Data\1,007 - Mail Transportation Inc v United States.doc,"HYPERLINK ""https://www.westlaw.com/Link/Docume...",0,0,0,0,0,0
1,C:\Users\muc574\Bid Protest\WestLaw Data\Data\1,008 - Mail Transportation Inc v United States.doc,"conversion serves the public interest, an inj...",1,0,0,0,1,0
2,C:\Users\muc574\Bid Protest\WestLaw Data\Data\1,010 - Account Control Technology Inc v United ...,a947fe08ef5e04eeedbf79e1eebe33c5c31feedea87c41...,0,0,0,0,0,0
3,C:\Users\muc574\Bid Protest\WestLaw Data\Data\1,011 - Van Ru Credit Corporation v United State...,5ad879fb8d2e86f4d2245807cb00009aa61f3e7c4892e4...,0,0,0,0,0,0
4,C:\Users\muc574\Bid Protest\WestLaw Data\Data\1,012 - Collection Technology Inc v United State...,686363039e5feb04f01211168160a6f6e8d123ad561b1c...,0,0,0,0,0,0


In [12]:
df.describe()

,contains_denied,contains_rejected,contains_dismissed,contains_remanded,total_unsuccessful,contains_settled
count,3404.000000,3404.000000,3404.000000,3404.000000,3404.000000,3404.000000
mean,0.454465,0.037015,0.159224,0.056110,0.706816,0.017920
std,0.497995,0.188827,0.365939,0.230169,0.683847,0.132681
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000


### Step 6 - ( '\\$' in paragraph and  'cost' in paragraph) or ( '\\$' in paragraph and  'contract' in paragraph)

In [13]:
#format 1
import os
import csv
from tqdm import tqdm
import time

def save_paragraphs_with_dollar_sign_to_csv(root_dir, csv_filename='paragraphs_with_dollar_format1.csv'):
    start_time = time.time()  # Start timing

    # Prepare to count files for progress bar
    total_files = sum([len(files) for r, d, files in os.walk(root_dir) if any(".doc" in f for f in files)])
    processed_files = 0

    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['subdir', 'Filename', 'Paragraph'])  # Write the header

        # Iterate through all subdirectories of the root directory
        for subdir, dirs, files in tqdm(os.walk(root_dir), total=total_files, desc="Processing files"):
            for filename in files:
                if ".doc" in filename:
                    file_path = os.path.join(subdir, filename)
                    try:
                        with open(file_path, 'r', encoding='utf-8') as file:
                            content = file.read()  # Read the entire file into a single string
                            paragraphs = content.split('\n')  # Split the content into paragraphs
                            for paragraph in paragraphs:
                                if ( '$' in paragraph and  'cost' in paragraph) or ( '$' in paragraph and  'contract' in paragraph) :
                                    #print(f"File: {filename}, Paragraph with $: {paragraph}")  # Optional: printing
                                    csvwriter.writerow([subdir, filename, paragraph])  # Save to CSV
                    except Exception as e:
                        print(f"Error reading file {filename}: {e}")
                    finally:
                        processed_files += 1
                        # Update progress manually if necessary
                        # tqdm.write(f"Processed {processed_files} / {total_files} files")

    elapsed_time = time.time() - start_time
    print(f"Completed in {elapsed_time:.2f} seconds")

# Define the root directory
root_directory = 'C:\\Users\\muc574\\Bid Protest\\WestLaw Data\\Data'

# Execute the function with the root directory as its argument
save_paragraphs_with_dollar_sign_to_csv(root_directory)


Processing files:   1%|          | 34/3405 [00:09<16:12,  3.47it/s]

Completed in 9.85 seconds


### Step 7 - Estimate the economic magnitude

In [22]:
import pandas as pd
import re

# Load the Excel file
file_path = 'paragraphs_with_dollar_format1.csv'
df = pd.read_csv(file_path)

# Step 1: Creating a unique identifier for each document
df['document_id'] = df['subdir'] + '\\' + df['Filename']

# Initialize columns for estimates and snippets
df['total_cost_estimate'] = "No Clear Cost Estimate"
df['contract_amount_estimate'] = "No Clear Contract Amount"
df['total_cost_snippet'] = ""
df['contract_amount_snippet'] = ""
df['per_item_or_unit_dummy'] = 0

# Define patterns for cost and contract amount identification
cost_pattern = re.compile(r'\$\d+(?:,\d{3})*(?:\.\d+)?(?: million| billion)?', re.IGNORECASE)
contract_amount_phrases = ['contract amount', 'contract sum', 'award']
total_cost_phrases = ['total cost', 'damages', 'cost', 'bid cost', 'attorney fee', 'labor cost', 'indirect cost', 'legal fee', 'fee']

# Function to convert formatted string values to numeric values
def convert_to_numeric(value):
    number = re.findall(r'\d+(?:,\d{3})*(?:\.\d+)?', value.replace(",", ""))
    if number:
        number = float(number[0].replace(",", ""))
        if 'million' in value.lower():
            return int(number) * 1e6
        elif 'billion' in value.lower():
            return int(number) * 1e9
        else:
            return int(number)
    return int(value)

# Iterate over rows to extract information
for index, row in df.iterrows():
    if "per item" in row['Paragraph'] or "per unit" in row['Paragraph']:
        df.at[index, 'per_item_or_unit_dummy'] = 1
    
    if any(phrase in row['Paragraph'].lower() for phrase in total_cost_phrases):
        matches = cost_pattern.findall(row['Paragraph'])
        if matches:
            # Convert and assign the numeric value
            numeric_value = convert_to_numeric(matches[0])
            df.at[index, 'total_cost_estimate'] = numeric_value
            df.at[index, 'total_cost_snippet'] = row['Paragraph'][:100]
    
    if any(phrase in row['Paragraph'].lower() for phrase in contract_amount_phrases):
        matches = cost_pattern.findall(row['Paragraph'])
        if matches:
            # Convert and assign the numeric value
            numeric_value = convert_to_numeric(matches[0])
            df.at[index, 'contract_amount_estimate'] = numeric_value
            df.at[index, 'contract_amount_snippet'] = row['Paragraph'][:100]

# Aggregating rows by document_id to ensure one row per document
aggregated_df = df.groupby('document_id').agg({
    'total_cost_estimate': 'last',
    'contract_amount_estimate': 'last',
    'per_item_or_unit_dummy': 'max',
    'total_cost_snippet': 'first',
    'contract_amount_snippet': 'first'
}).reset_index()

# Exporting the aggregated data to an Excel file
output_path = 'output_1.xlsx'
aggregated_df.to_excel(output_path, index=False)

print(f"Data has been successfully aggregated and exported to {output_path}.")


Data has been successfully aggregated and exported to output_1.xlsx.


In [23]:
from scipy.stats.mstats import winsorize
import pandas as pd
import numpy as np
import warnings

# Suppress specific warnings from pandas
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

# Assuming aggregated_df is already defined

# Make a copy of aggregated_df to avoid changing the original DataFrame
df = aggregated_df.copy()

# Convert 'total_cost_estimate' from string to numeric, setting errors='coerce' to turn non-convertible values into NaN
df['total_cost_estimate'] = pd.to_numeric(df['total_cost_estimate'], errors='coerce')
df['contract_amount_estimate'] = pd.to_numeric(df['contract_amount_estimate'], errors='coerce')

# Replace missing 'total_cost_estimate' with 'contract_amount_estimate' where applicable
df.loc[df['total_cost_estimate'].isnull() & df['contract_amount_estimate'].notnull(), 'total_cost_estimate'] = df['contract_amount_estimate']

# Filter rows where 'total_cost_estimate' is not NaN (after replacement)
df_1 = df[df['total_cost_estimate'].notnull()].copy()

# Ensure 'contract_amount_estimate' is numeric (should already be, but this is to double-check)
df_1['contract_amount_estimate'] = pd.to_numeric(df_1['contract_amount_estimate'], errors='coerce')

# Add logged variables of the numeric columns
df_1['logged_total_cost_estimate'] = np.log1p(df_1['total_cost_estimate'])
df_1['logged_contract_amount_estimate'] = np.log1p(df_1['contract_amount_estimate'])

# Winsorize the numeric columns at 2.5% for both tails
df_1['total_cost_estimate_winsorized'] = winsorize(df_1['total_cost_estimate'], limits=[0.05, 0.05])
df_1['contract_amount_estimate_winsorized'] = winsorize(df_1['contract_amount_estimate'], limits=[0.05, 0.05])

# Winsorize the logged columns
df_1['logged_total_cost_estimate_winsorized'] = winsorize(df_1['logged_total_cost_estimate'], limits=[0.05, 0.05])
df_1['logged_contract_amount_estimate_winsorized'] = winsorize(df_1['logged_contract_amount_estimate'], limits=[0.05, 0.05])

# Keep only the relevant columns
columns_to_keep = ['logged_total_cost_estimate_winsorized']
df_1 = df_1[columns_to_keep]

# Describe the winsorized and logged variables
description = df_1.describe()

# Convert to more readable format by rounding and using applymap to avoid scientific notation
description_readable = description.round(2).applymap(lambda x: '{:,}'.format(x))

description_readable


,logged_total_cost_estimate_winsorized
count,"1,113.0"
mean,12.46
std,4.48
min,2.83
25%,9.62
50%,12.75
75%,15.89
max,19.79
